In [124]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split



from sklearn.metrics import confusion_matrix, r2_score, recall_score, precision_score, f1_score, accuracy_score

In [137]:
cd 

/Users/rashidbaset


In [141]:
# CLEANED WATER SET

df = pd.read_csv("..data/cleaned_water.csv",index_col=0)

#define classes
df['status_group'] = df['status_group'].replace(to_replace =["functional needs repair", "non functional"],value ="needs repair")
df['status_group'] = df['status_group'].map({'functional': 0, 'needs repair': 1})

df.head()

FileNotFoundError: [Errno 2] File b'..data/cleaned_water.csv' does not exist: b'..data/cleaned_water.csv'

In [139]:
#(df == 0).sum()

In [84]:
#if we want to have a binary classification problem:
# df['status_group'] = df['status_group'].replace(to_replace =["functional needs repair", "non functional"],
#                             value ="needs repair")

###### **if we were to use get_dummies()...**

In [ ]:
# # df_num = df[['amount_tsh', 'population', 'gps_height']]
# df_cat = df[['funder', 'scheme_management', 'extraction_type_group', 'management', 'water_quality', 'quantity', 'installer', 'basin']]

In [ ]:
# df_dummies = pd.get_dummies(df_cat, drop_first=True)

# df_final = pd.concat([df_num, df_dummies], axis=1)
# df_final.shape

## OHE

In [ ]:
# # le = LabelEncoder()
# # le.fit(df['status_group'])
# df['status_group'] = pd.DataFrame(le.transform(df['status_group']),columns=['status_group'])
# # df['status_group'] = df.status_group.astype('int64')

In [ ]:
df.head()

In [15]:
# create y and X as the target and the features
y = df['status_group']
X = df.drop(columns='status_group')

In [16]:
y

0        0
2        0
3        1
5        0
6        1
        ..
59394    1
59395    0
59396    0
59398    0
59399    0
Name: status_group, Length: 48651, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48651 entries, 0 to 59399
Data columns (total 22 columns):
status_group             48651 non-null int64
amount_tsh               48651 non-null float64
funder                   48651 non-null object
gps_height               48651 non-null int64
installer                48651 non-null object
basin                    48651 non-null object
region                   48651 non-null object
lga                      48651 non-null object
ward                     48651 non-null object
population               48651 non-null int64
public_meeting           48651 non-null bool
scheme_management        48651 non-null object
permit                   48651 non-null bool
extraction_type_class    48651 non-null object
management               48651 non-null object
payment_type             48651 non-null object
water_quality            48651 non-null object
quantity                 48651 non-null object
source                   48651 non-null object
sour

In [18]:
cat_cols = X.select_dtypes(include=['object','bool']).columns.tolist()
cont_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols,cont_cols

(['funder',
  'installer',
  'basin',
  'region',
  'lga',
  'ward',
  'public_meeting',
  'scheme_management',
  'permit',
  'extraction_type_class',
  'management',
  'payment_type',
  'water_quality',
  'quantity',
  'source',
  'source_class',
  'waterpoint_type'],
 ['amount_tsh', 'gps_height', 'population', 'age'])

In [61]:
for col in cat_cols:
    X[col] = X[col].astype('category')

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 77, 
                                                    stratify = y, # in classification problems 
                                                                  # when you split the data 
                                                                  # you want to keep the ratio in the classes.
                                                    test_size = .2 # This is usually the ratio but it might change 
                                                                   # according to the problem at hand.
                                                   )

In [21]:
## create an encoder object. This will help us to convert
## categorical variables to new columns
ss = StandardScaler()
encoder = OneHotEncoder(handle_unknown= 'ignore',
                        #drop='first',
                        categories= 'auto')

## Create an columntransformer object.
## This will help us to merge transformed columns
## with the rest of the dataset.

ct = ColumnTransformer(transformers =[('ohe', encoder, cat_cols)], #('scaler', ss, cont_cols)],
                                    remainder= ss)
X_train_ohe = ct.fit_transform(X_train)
X_test_ohe = ct.transform(X_test)

In [22]:
#cats_ohe 

In [23]:
#cats_ohe = ct.get_feature_names()

In [24]:
X_train_ohe.shape, X_test_ohe.shape

((38920, 5134), (9731, 5134))

In [25]:
y_train.shape,y_test.shape

((38920,), (9731,))

---

#### Dummy Modeling

- Linear Regression
- Random Forest
- xboost


logistic, random forest, xboost, (confusion, class report, metric (top 2 or 3) hypter par tune top models. 2 adequate. which one is more simplistic if approx same value.)

In [26]:
1 - y.mean()

0.5492178989126637

#### Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='liblinear', penalty='l1')
log_reg.fit(X_train_ohe, y_train)
log_reg.score(X_train_ohe, y_train)

0.8248715313463515

In [35]:
y_pred = log_reg.predict(X_train_ohe)

In [33]:
import matplotlib.pyplot as plt

# From https://scikit-learn.org/0.18/auto_examples/model_selection/plot_confusion_matrix.html

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [34]:
cnf_matrix = confusion_matrix(y_test, y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [9731, 38920]

In [68]:
# Vanilla KNN
knn =KNeighborsClassifier()
knn.fit(X_train_ohe, y_train)
knn.score(X_test_ohe, y_test)

0.7996094954269859

In [45]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_ohe, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [46]:
# make class predictions for the testing set
y_pred_class = knn.predict(X_test_ohe)

In [47]:
from sklearn import metrics
print('Recall:' + str(metrics.recall_score(y_test, y_pred_class)))
print('F1: ' + str(metrics.f1_score(y_test, y_pred_class)))

from sklearn.metrics import classification_report  
print(classification_report(y_test, y_pred_class))

Recall:0.733758832915432
F1: 0.7675250357653791
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      5344
           1       0.80      0.73      0.77      4387

    accuracy                           0.80      9731
   macro avg       0.80      0.79      0.80      9731
weighted avg       0.80      0.80      0.80      9731



#### RandomForest Classifier

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
clf = RandomForestClassifier(n_estimators= 1000, 
                             criterion= 'gini', 
                             max_features= 'sqrt',
                             max_depth= 4,
                             min_samples_leaf= 6,
                             oob_score= True,
                             n_jobs = -1)

In [54]:
clf.fit(X_train_ohe, y_train)
clf.score(X_train_ohe, y_train)
clf.oob_score_

0.6656217882836588

In [55]:
feature_importances = clf.feature_importances_

In [79]:
columns = ohe.get_feature_names().tolist() +numerical_variables.tolist()

NameError: name 'ohe' is not defined

In [78]:
columns

NameError: name 'columns' is not defined

In [ ]:
importances = pd.DataFrame(data= feature_importances, index = columns, columns= ['feature_importances'])

importances.sort_values(by = 'feature_importances', ascending = False)

### Decision Tree

In [58]:
from sklearn.tree import DecisionTreeClassifier

In [59]:
clf = DecisionTreeClassifier()

In [64]:
clf.fit(X_train_ohe,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [65]:
clf.score(X_train_ohe, y_train)

0.9745118191161357

In [66]:
recall_score(y_train, y_pred)

0.7502279981760146

In [67]:
f1_score(y_train, y_pred)

0.7943270971635487

In [74]:
X_train_ohe

<38920x5134 sparse matrix of type '<class 'numpy.float64'>'
	with 817320 stored elements in Compressed Sparse Row format>

In [72]:
cols = X_train_ohe.columns

AttributeError: columns not found

In [75]:
cols = pd.DataFrame(X_train_ohe)

In [77]:
cols

,0
0,"(0, 1301)\t1.0\n (0, 2724)\t1.0\n (0, 3048..."
1,"(0, 401)\t1.0\n (0, 1727)\t1.0\n (0, 3049)..."
2,"(0, 847)\t1.0\n (0, 1727)\t1.0\n (0, 3045)..."
3,"(0, 413)\t1.0\n (0, 1896)\t1.0\n (0, 3049)..."
4,"(0, 981)\t1.0\n (0, 2555)\t1.0\n (0, 3050)..."
...,...
38915,"(0, 131)\t1.0\n (0, 1727)\t1.0\n (0, 3050)..."
38916,"(0, 1355)\t1.0\n (0, 1727)\t1.0\n (0, 3051..."
38917,"(0, 1355)\t1.0\n (0, 1727)\t1.0\n (0, 3051..."
38918,"(0, 1334)\t1.0\n (0, 1706)\t1.0\n (0, 3049..."


In [142]:
feature_importances = clf.feature_importances_
columns = ohe.get_feature_names().tolist() +numerical_variables.tolist()

importances = pd.DataFrame(data= feature_importances, index = columns, columns= ['feature_importances'])

importances.sort_values(by = 'feature_importances', ascending = False)

# Top 3 highest feature importance weight
cols = pd.DataFrame(X_train_ohe)
importances = pd.DataFrame({'feature':cols.columns,'importance':np.round(clf.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False)

ValueError: arrays must all be same length

## Metrics

#### Confusion Matrix